# Extraindo dados da Covid19 via API
### Fonte de Dados: https://brasil.io/dataset/covid19/
### Augusto SPINELLI

### Licença
Os dados dados convertidos estão sob a licença Creative Commons Attribution ShareAlike. Caso utilize os dados, cite a fonte original e quem tratou os dados, como: Fonte: Secretarias de Saúde das Unidades Federativas, dados tratados por Álvaro Justen e colaboradores/Brasil.IO. Caso compartilhe os dados, utilize a mesma licença.

***

## Exploração de Dados

In [1]:
#importando dependencias
import pandas as pd
import numpy as np
import os

#Caminho dos arquivos RAW
raw_data_path = os.path.join(os.path.pardir, 'data', 'raw')
estados_data_path = os.path.join(raw_data_path, 'estados.json')
cidades_data_path = os.path.join(raw_data_path, 'cidades.json')

#criando dataframes brutos/raw
cidadesRaw_df = pd.read_json(cidades_data_path)
estadosRaw_df = pd.read_json(estados_data_path)

In [2]:
#transformando dados para dataframes finais com orientação correta
cidades = cidadesRaw_df['results']
cidades_df = pd.read_json(cidades.to_json(),orient='index')

estados = estadosRaw_df['results']
estados_df = pd.read_json(estados.to_json(),orient='index')

In [4]:
#verificando dataframes
cidades_df.info()
estados_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5150 entries, 0 to 5149
Data columns (total 12 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   city                            5150 non-null   object        
 1   city_ibge_code                  5133 non-null   float64       
 2   confirmed                       5150 non-null   int64         
 3   confirmed_per_100k_inhabitants  5068 non-null   float64       
 4   date                            5150 non-null   datetime64[ns]
 5   death_rate                      5150 non-null   float64       
 6   deaths                          5150 non-null   int64         
 7   estimated_population_2019       5133 non-null   float64       
 8   is_last                         5150 non-null   bool          
 9   order_for_place                 5150 non-null   int64         
 10  place_type                      5150 non-null   object        
 11  stat